# Homework 7

1. Following our discussion/confusion on the `r2_score`: think about the shortcomings of using the squared correlation coefficient as a quality of fit measure. Construct a few simple examples, that highlight, why `np.corrcoef(y_test,yHat)**2` would be a very poor choice.

2. Look up the sklearn help file for [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html#sklearn.preprocessing.PolynomialFeatures) and translate the statsmodel from below into the corresponding sklearn version, using `LinearRegression()` only.
3. Look up the `patsy` help file for [dmatrix](https://patsy.readthedocs.io/en/latest/formulas.html#redundancy-and-categorical-factors) and translate the statsmodel from below into the corresponding sklearn version, using `LinearRegression()` only.

4. Estimate the *10-fold* CV error on this model.



**Importing libraries**

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
#import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy as ps
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

## Task 1: Shortcoming of using R²

In [210]:
#R² can simply be overloaded by adding predictors. 
#No matter how significant the predictor actually is, the R² will increase.
#Therefore, I will revisit our excersice of purely random significant predictors.
XY = np.random.rand(100,96)
col_names = ["Y"]
col_names += ["x_"+str(i) for i in range(1,96)]
col_names

df = pd.DataFrame(XY, columns = col_names)

x_names = col_names[1:]
formula = "Y ~ "+("+").join(x_names)
formula
est = smf.ols(formula, df).fit()
est.summary().tables[0]

Dep. Variable:,Y,R-squared:,0.927
Model:,OLS,Adj. R-squared:,-0.817
Method:,Least Squares,F-statistic:,0.5313
Date:,"Sun, 21 Nov 2021",Prob (F-statistic):,0.880
Time:,09:44:43,Log-Likelihood:,116.41
No. Observations:,100,AIC:,-40.82
Df Residuals:,4,BIC:,209.3
Df Model:,95,,
Covariance Type:,nonrobust,,


This example shows, thats by creating 95 random variables of each length 100, we can achieve an almost R². As a matter of fact, if we further increase the number of variables we will be at a point, where we consistently find an R² of 1.00.

Interestingly, the adjusted R² seems somewhat powerless for this example, given that from multiple tries I found adjusted R² between -2.6 (!) and 0.85

In [226]:
auto = pd.read_csv(r"C:\Users\svawe\OneDrive\Desktop\Master\DataScience_01\data\Auto.csv")
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,Manufacturer
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320,buick
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite,plymouth
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst,amc
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino,ford


In [224]:
#Next, lets have a look at doing the same with categorical variables ...
model = smf.ols('mpg ~ cylinders + displacement + horsepower + weight + acceleration + year + origin +C(Manufacturer)', auto).fit()
model.summary().tables[0]

Dep. Variable:,mpg,R-squared:,0.842
Model:,OLS,Adj. R-squared:,0.826
Method:,Least Squares,F-statistic:,52.66
Date:,"Sun, 21 Nov 2021",Prob (F-statistic):,2.59e-120
Time:,09:54:41,Log-Likelihood:,-999.19
No. Observations:,392,AIC:,2072.
Df Residuals:,355,BIC:,2219.
Df Model:,36,,
Covariance Type:,nonrobust,,


In [225]:
model = smf.ols('mpg ~ C(cylinders) + C(displacement) + C(horsepower) + weight + C(acceleration) + C(year) + C(origin) +C(Manufacturer)', auto).fit()
model.summary().tables[0]

Dep. Variable:,mpg,R-squared:,0.982
Model:,OLS,Adj. R-squared:,0.931
Method:,Least Squares,F-statistic:,19.35
Date:,"Sun, 21 Nov 2021",Prob (F-statistic):,7.00e-44
Time:,09:55:14,Log-Likelihood:,-575.35
No. Observations:,392,AIC:,1729.
Df Residuals:,103,BIC:,2876.
Df Model:,288,,
Covariance Type:,nonrobust,,


The R² can be increased from 0.842 to 0.982 by just converting several variables to categorial variables.

Yet again, the adjusted R² is failing too, increasing by almost the same amount from 0.826 to 0.931.

In [241]:
#... and interactions!
model = smf.ols('mpg ~ C(cylinders)*C(displacement) + C(horsepower)*weight + C(acceleration) + C(year) + C(origin)*C(Manufacturer)', auto).fit()
model.summary().tables[0]

Dep. Variable:,mpg,R-squared:,0.990
Model:,OLS,Adj. R-squared:,0.933
Method:,Least Squares,F-statistic:,17.23
Date:,"Sun, 21 Nov 2021",Prob (F-statistic):,6.13e-25
Time:,10:33:40,Log-Likelihood:,-458.69
No. Observations:,392,AIC:,1585.
Df Residuals:,58,BIC:,2912.
Df Model:,333,,
Covariance Type:,nonrobust,,


There goes the 0.99, by just adding another interaction we could easily achieve 1.00. The adjusted R² still seems better, however, with 0.933 it is still very very high.

# The beging of the rest of the tasks

**Loading the dataframe**

In [52]:
boston = pd.read_csv(r"C:\Users\svawe\OneDrive\Desktop\Master\DataScience_01\data\Boston.csv")

#X = boston.drop('medv', axis=1)
y = boston['medv'].values
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [53]:
model = smf.ols('medv ~ lstat*rm + C(chas) + crim ', boston).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-29.8113,3.268,-9.121,0.000,-36.233,-23.390
C(chas)[T.1],3.2168,0.803,4.006,0.000,1.639,4.795
lstat,2.2458,0.201,11.170,0.000,1.851,2.641
rm,9.6980,0.489,19.843,0.000,8.738,10.658
lstat:rm,-0.4818,0.034,-14.346,0.000,-0.548,-0.416
crim,-0.1138,0.027,-4.278,0.000,-0.166,-0.062


## Task 2

In [85]:
boston["lstat*rm"] = boston.lstat * boston.rm  
X = boston[["chas","lstat","rm","lstat*rm","crim"]].values
poly = PolynomialFeatures(degree=1) #I think this is not what you are going for, but I dont really get it.
X = poly.fit_transform(X)
regr = LinearRegression().fit(X, y)
yHat = regr.intercept_+ np.dot(X,regr.coef_)
print(f"the intercept is: {regr.intercept_}, the regression coefficients are {regr.coef_}")

the intercept is: -29.8113251820791, the regression coefficients are [ 0.          3.21678536  2.24584117  9.69798331 -0.48180729 -0.11376058]


## Task 3

In [72]:
boston["lstat*rm"] = boston.lstat * boston.rm  
y, X = ps.dmatrices("medv ~ lstat*rm + C(chas) + crim", boston)
regr = LinearRegression().fit(X,y)
print(f"the intercept is: {regr.intercept_}, the regression coefficients are {regr.coef_}")

the intercept is: [-29.81132518], the regression coefficients are [[ 0.          3.21678536  2.24584117  9.69798331 -0.48180729 -0.11376058]]


## Task 4

In [77]:
X = boston[["chas","lstat","rm","lstat*rm","crim"]].values
y = boston['medv'].values

regr = LinearRegression()
cv_results = cross_val_score(regr, X, y, cv=10, scoring = "neg_mean_squared_error")

print("CV results: ", np.sqrt(abs(cv_results)))
print("CV results mean: ", np.mean(np.sqrt(abs(cv_results))))

CV results:  [ 2.89147289  2.62797553  2.50034179  5.35999789  3.76235528  4.79248786
  3.91664707 11.17815848  3.84310694  3.66934068]
CV results mean:  4.454188441597394
